In [1]:
import gym
from gym import spaces
import numpy as np

class TrafficGenerator(gym.Env):
    
    #doesnt really workj
    def getObs(self):
        # Loop over the packet types
        for i in range(len(self.mean_delay)):

            # Get the current queue for the packet type
            current_queue = self.queues[i]

            # Calculate the length and average waiting time of the current queue
            current_length = len(current_queue)
            current_waiting_time = np.average(current_queue) if current_length > 0 else 0.0
            print(np.average(current_queue), current_length, current_waiting_time )
            
            
           
            observation.append([current_length, current_waiting_time])

       
        observation = np.array(observation)
        return observation
    
    def __init__(self):
        
        self.action_space = spaces.Discrete(3)
        
        # Define the observation space (number of packets in each queue and their waiting time)
        self.observation_space = spaces.Box(low=0, high=np.inf, shape=(3,2), dtype=int)
        
        # pancket info (DataType, arrival_rate, mean_delay )
        #self.packetInfo = [[0, 0.3, 6],[1, 0.25, 4],[2, 0.4, float('inf')]]
        
        self.dataType = [0, 1, 2]
        self.arrival_rate = [0.3, 0.25, 0.4]
        self.mean_delay = [6, 4, float('inf')]
        self.packetInfo = [[elem1, elem2, elem3] for elem1, elem2, elem3 in zip(self.dataType, self.arrival_rate, self.mean_delay)]
        
        self.packet = 1
        self.timeslot = 1
        
        # Initialize the queues
        self.queues = [[], [], []]
      
    
    def step(self, action):  
        #packet generator
        for i in range(len(self.packetInfo)):
            for sublist in self.packetInfo:
                if sublist[0] == i and np.random.uniform() < sublist[1]:
                    self.queues[i].append(self.packet)
                    print("Appending to queue", i)
        
        for i in range(len(self.queues)):
            for j in range(len(self.queues[i])):
                self.queues[i][j] -= self.timeslot
                
        
        # services the queues
        for i in range(len(self.queues)):
            print("queue ", i, ": has a length of", len(self.queues[i]))
        print("ACTION queue ", action, ": has a length of", len(self.queues[action]))
        if action < 3 and len(self.queues[action]) > 0:
            self.queues[action].pop(0)
            for i in range(len(self.queues[action])):
                self.queues[action][i] -= 1
        

        
        #TODO rewardoptions?
        
         # Loop over the packet types
        observation = []
        for i in range(len(self.mean_delay)):

            # Get the current queue for the packet type
            current_queue = self.queues[i]

            # Calculate the length and average waiting time of the current queue
            current_length = len(current_queue)
            current_waiting_time = np.average(current_queue) if current_length > 0 else 0.0
            #print(np.average(current_queue), current_length, current_waiting_time )
            
            
    
            observation.append([current_length, current_waiting_time])

        observation = np.array(observation)
        
        done = len(self.queues[0]) + len(self.queues[1]) + len(self.queues[2]) == 0
        return observation, done
        
    def reset(self):
        # Reset the queues
        self.queues = [[], [], []]
        return np.array([[len(self.queues[i]), 0.0] for i in range(len(self.mean_delay))])
        
    def render(self):
        pass

In [2]:
env = TrafficGenerator()
obs = env.reset()
done = False
x = 0
while x < 100:
    print("~~~~~~~~~")
    print(obs[:, 0])
    
    if obs[:, 0][0] > 0 or obs[:, 0][1] > 0:
        action = np.argmax(obs[:2, 0])
    else:
        action = 2 
    
    print("chosen", action)
    obs, done = env.step(action)
    print(obs)
    x = x +1

~~~~~~~~~
[0. 0. 0.]
chosen 2
Appending to queue 0
Appending to queue 1
queue  0 : has a length of 1
queue  1 : has a length of 1
queue  2 : has a length of 0
ACTION queue  2 : has a length of 0
[[1. 0.]
 [1. 0.]
 [0. 0.]]
~~~~~~~~~
[1. 1. 0.]
chosen 0
Appending to queue 0
queue  0 : has a length of 2
queue  1 : has a length of 1
queue  2 : has a length of 0
ACTION queue  0 : has a length of 2
[[ 1. -1.]
 [ 1. -1.]
 [ 0.  0.]]
~~~~~~~~~
[1. 1. 0.]
chosen 0
Appending to queue 0
queue  0 : has a length of 2
queue  1 : has a length of 1
queue  2 : has a length of 0
ACTION queue  0 : has a length of 2
[[ 1. -1.]
 [ 1. -2.]
 [ 0.  0.]]
~~~~~~~~~
[1. 1. 0.]
chosen 0
queue  0 : has a length of 1
queue  1 : has a length of 1
queue  2 : has a length of 0
ACTION queue  0 : has a length of 1
[[ 0.  0.]
 [ 1. -3.]
 [ 0.  0.]]
~~~~~~~~~
[0. 1. 0.]
chosen 1
Appending to queue 2
queue  0 : has a length of 0
queue  1 : has a length of 1
queue  2 : has a length of 1
ACTION queue  1 : has a length of 1


Training the DQN agent dude

In [3]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam

In [4]:
states = env.observation_space.shape
actions = env.action_space.n

In [36]:
def build_model(states, actions):
    model = Sequential() 
    model.add(Flatten(input_shape=states))
    model.add(Dense(24, activation='relu', input_shape=states))
    model.add(Dense(24, activation='relu'))
    model.add(Dense(actions, activation='linear'))
    model.compile(optimizer=Adam(), loss='mse')

    return model

In [37]:
model = build_model(states, actions)


In [7]:
states

(3, 2)

In [8]:
from rl.agents import DQNAgent
from rl.policy import BoltzmannQPolicy
from rl.memory import SequentialMemory

In [42]:
def build_agent(model, actions, states):
    policy = BoltzmannQPolicy()
    memory = SequentialMemory(limit=50000, window_length=1)
    dqn = DQNAgent(model=model, memory=memory, policy=policy, 
                  nb_actions=actions, nb_steps_warmup=10, target_model_update=1e-2)
    return dqn

In [53]:
optimizer = Adam(learning_rate=1e-3)
Adam._name = "adam"
dqn = build_agent(model, actions, states)
dqn.compile(optimizer, metrics=['mae'])
dqn.fit(env, nb_steps=50000, visualize=False, verbose=1)

Training for 50000 steps ...
Interval 1 (0 steps performed)


ValueError: Error when checking input: expected flatten_2_input to have 3 dimensions, but got array with shape (1, 1, 3, 2)

In [41]:

print(tuple())


3


In [15]:
print(states)


(3, 2)
